### Import and set-up

In [5]:
from dataclasses import dataclass, field
from typing import List, Tuple, Any

In [ ]:
Instruction = Tuple[str, Any]

# create boilerplate for dataclass CPU
# dataclass decorator creates class boilerplates that already have __init__, __repr__, and __eq__ defined
@dataclass
class CPU:
    mem: List[int] = field(default_factory=lambda: [0]*32)      # 32 integer cells of RAM
    acc: int = 0                                                # Accumulator
    idx: int = 0                                                # Index register
    pc: int = 0                                                 # Program counter
    halted: bool = False
    program: List[Instruction] = field(default_factory=list)    # program is a List of Instructions (list of tuples)

    # load program, with starting values for each field except mem
    def load_program(self, program: List[Instruction]):
        '''Load a list of instructions and reset CPU state.'''
        self.program = program
        self.pc = 0
        self.halted = False
        self.acc = 0
        self.idx = 0
        # memory persists until you overwrite cpu.mem explicitly

    def dump(self):
        '''Snapshot of key state for quick inspection'''
        return{
            'PC': self.pc,
            'ACC': self.acc,
            'IDX': self.idx,
            'HALTED': self.halted,
            'MEMO..7': self.mem[:8]
        }
    
    def step(self):
        '''Execute exactly the next instruction'''
        # if halted or pc is not between 0 and len(program), set halted = True
        if self.halted or not (0 <= self.pc <= len(self.program)):
            self.halted = True 
            return
        
        # reset
        target = None
        target_index = None
        off = None
        off_target = None
        addr = None

        instr = self.program[self.pc]
        # unpack instruction into op and *args or 
        op, *args = instr if isinstance(instr, tuple) else (instr,)
        self.pc += 1

        # enforce that *args[0] is int
        if len(args) >= 1 and not isinstance(args[0], int):
            raise TypeError(f'({op}, {args}): second tuple item must be an int')

        # sets of similar instructions
        uses_addr = {'STORE_ABS', 'LOAD_ABS', 'ADD_ABS'}
        uses_off = {'LOAD_IDX_OFF', 'ADD_IDX_OFF'}

        # set addr
        def set_addr
        op_uses_addr = op in uses_addr.union(uses_off)
        if op_uses_addr:
            if not args:
                raise ValueError(f'op {op} must have memory address.')
            addr = args[0]
            if not(0 <= addr < len(self.mem)):
                raise IndexError(f'({op}, {args}) contains a bad memory address.')
        
        # set offset
        op_uses_offset = op in uses_off
        if op_uses_offset:
            if not args:
                raise ValueError(f'op {op} must have offset.')
            off = args[1]
            off_target = self.idx + off
            if not (0 <= off_target < len(self.mem)):
                raise IndexError(f'({op}, {args}) results in a bad memory address.')

        def _check_idx(self):
            if not (0 <= self.idx < len(self.mem)):
                raise IndexError(f'IDX out of range: {self.idx}')
        
        # check for inc/dec
        def _check_target_idx(self):
            if 


        if op == 'LOADI':           # ACC <- immediate
            self.acc = args[0]
        elif op == 'LOAD_ABS':      # ACC <- mem[addr]
            # good address enforced in set addr
            self.acc = self.mem[addr]
        elif op == 'LOAD_IDX':      # ACC <- mem[idx]
            _check_idx()
            self.acc = self.mem[self.idx]
        elif op == 'LOAD_IDX_OFF':  # ACC <- mem[idx+off]
            target_index = off_target
            self.ACC = self.mem[target_index]
        elif op == 'STORE_ABS':     # mem[addr] <- ACC
            self.mem[addr] = self.acc
        elif op == 'STORE_IDX':     # mem[idx] <- ACC
            _check_idx()
            self.mem[self.idx] = self.acc
        elif op == 'STORE_IDX_OFF':  # mem[idx+off] <- ACC
            target_index = off_target
            self.mem[target_index] = self.acc
        elif op == 'ADDI':          # acc = acc + immediate
            self.acc = self.acc + args[0]
        elif op == 'ADD_ABS':       # acc = acc + mem[addr]
            self.acc = self.acc + self.mem[addr]
        elif op == 'ADD_IDX':       # acc = acc + mem[idx]
            _check_idx()
            self.acc += self.mem[self.idx]
        elif op == 'INC_IDX':       
            self.idx += 1
        elif op == 'DEC_IDX':
            self.idx -= 1
        elif op == 'JUMP_ABS':      # Absolute jump
            target = int(args[0])
            if not (0 <= target < len(self.program)):
                raise IndexError(f'Bad pc: {target}')
            self.pc = target
        elif op == 'JUMP_REL':      # Relative jump
            target = self.pc + int(args[0])
            if not (0 <= target < len(self.program)):
                raise IndexError(f'Bad pc: {target}')
            self.pc = target
        elif op == 'HALT':          # stop execution
            self.halted = True
        else:
            raise NotImplementedError(f'Unknown op: {op}')        
    
    def run(self, max_steps=100):
        '''Run up to max_steps or until halted.'''
        steps = 0
        while not self.halted and steps < max_steps:
            self.step()
            steps += 1
        return steps


In [36]:
prog = [
    ('LOADI', -7),
    ('STORE_ABS', 0),
    ('LOADI', 12),
    ('STORE_ABS', 1),
    ('LOAD_ABS', 1),
    ('HALT',),
]

In [37]:
cpu = CPU()
cpu.load_program(prog)
cpu.run()
print(cpu.dump())


{'PC': 6, 'ACC': 12, 'IDX': 0, 'HALTED': True, 'MEMO..7': [-7, 12, 0, 0, 0, 0, 0, 0]}


In [41]:
args = [3, 4, 'bobi']

all([isinstance(arg, int) for arg in args])

False